In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [3]:
import string

In [5]:
import os
import cv2
import string
import torch
import pandas as pd
from tqdm import tqdm
from torch import nn
from ultralytics import YOLO

# ==== CONFIG ====
IMG_WIDTH, IMG_HEIGHT = 160, 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CHARS = string.digits + string.ascii_uppercase + 'T'
CHAR_TO_IDX = {c: i + 1 for i, c in enumerate(CHARS)}
IDX_TO_CHAR = {i: c for c, i in CHAR_TO_IDX.items()}

# ==== CRNN MODEL ====
class CRNN(nn.Module):
    def __init__(self, num_classes):
        super(CRNN, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU(), nn.MaxPool2d((2, 1)),
            nn.Conv2d(256, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.MaxPool2d((2, 1)),
        )
        self.rnn = nn.LSTM(1024, 256, bidirectional=True, num_layers=2, batch_first=True)
        self.fc = nn.Linear(512, num_classes + 1)

    def forward(self, x):
        x = self.cnn(x)
        b, c, h, w = x.size()
        x = x.permute(0, 3, 1, 2).contiguous().view(b, w, c * h)
        x, _ = self.rnn(x)
        x = self.fc(x[:, :, :512])
        return x.permute(1, 0, 2)

# ==== Decode ====
def decode_prediction(logits):
    pred = torch.argmax(logits, dim=2).permute(1, 0)
    results = []
    for seq in pred:
        prev = -1
        text = ""
        for idx in seq:
            idx = idx.item()
            if idx != prev and idx != 0:
                text += IDX_TO_CHAR.get(idx, "?")
            prev = idx
        results.append(text)
    return results

In [6]:
# ==== Make sure these are set ====
YOLO_MODEL_PATH = "/content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/plate_detector_v2/weights/best.pt"
MODEL_SAVE_PATH = "/content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/ocr_crnn_best.pth"
FULL_CAR_IMG_DIR = "/content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/test/images"
SAVE_CSV_PATH = "/content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/all_predictions.csv"

IMG_WIDTH, IMG_HEIGHT = 160, 32
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CHARS = string.digits + string.ascii_uppercase + 'T'
IDX_TO_CHAR = {i: c for c, i in {c: i+1 for i, c in enumerate(CHARS)}.items()}

# ==== Load Models ====
yolo_model = YOLO(YOLO_MODEL_PATH)

crnn_model = CRNN(len(CHARS)).to(DEVICE)
crnn_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=DEVICE))
crnn_model.eval()

# ==== Predict Function ====
def predict_text_from_image(img_crop):
    gray = cv2.cvtColor(img_crop, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (IMG_WIDTH, IMG_HEIGHT))
    norm = resized / 255.0
    input_tensor = torch.tensor(norm, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        logits = crnn_model(input_tensor)
    return decode_prediction(logits)[0]

# ==== Run Inference on All Images ====
results = []
test_images = sorted([f for f in os.listdir(FULL_CAR_IMG_DIR) if f.lower().endswith(('.jpg', '.png'))])

for img_name in tqdm(test_images, desc="🔍 Predicting plates"):
    img_path = os.path.join(FULL_CAR_IMG_DIR, img_name)
    result = yolo_model(img_path)
    img = cv2.imread(img_path)
    boxes = result[0].boxes.xyxy.cpu().numpy()

    if len(boxes) == 0:
        results.append({'img_id': img_name, 'text': 'No plate detected'})
        continue

    x1, y1, x2, y2 = map(int, boxes[0][:4])  # First detected plate only
    cropped = img[y1:y2, x1:x2]
    text = predict_text_from_image(cropped)
    results.append({'img_id': img_name, 'text': text})

# ==== Save to Drive ====
df = pd.DataFrame(results)
df.to_csv(SAVE_CSV_PATH, index=False)
print(f"✅ Saved all predictions to: {SAVE_CSV_PATH}")


🔍 Predicting plates:   0%|          | 0/210 [00:00<?, ?it/s]


image 1/1 /content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/test/images/1000.jpg: 448x640 1 license_plate, 333.9ms
Speed: 16.2ms preprocess, 333.9ms inference, 29.1ms postprocess per image at shape (1, 3, 448, 640)


🔍 Predicting plates:   0%|          | 1/210 [00:09<31:48,  9.13s/it]


image 1/1 /content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/test/images/1001.jpg: 640x384 1 license_plate, 155.1ms
Speed: 2.7ms preprocess, 155.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)


🔍 Predicting plates:   1%|          | 2/210 [00:09<14:22,  4.14s/it]

...


image 1/1 /content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/test/images/998.jpg: 640x480 1 license_plate, 162.7ms
Speed: 2.9ms preprocess, 162.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)


🔍 Predicting plates: 100%|█████████▉| 209/210 [01:10<00:00,  3.91it/s]


image 1/1 /content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/test/images/999.jpg: 384x640 1 license_plate, 139.1ms
Speed: 2.9ms preprocess, 139.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


🔍 Predicting plates: 100%|██████████| 210/210 [01:10<00:00,  2.96it/s]

✅ Saved all predictions to: /content/drive/MyDrive/DATA_SCIENTIST_ASSIGNMENT/all_predictions.csv
